In [74]:
import numpy as np
import pandas as pd
from tqdm import tqdm

ds = pd.read_json('quotes-2019-nytimes.json', lines=True)

In [14]:
# just for the purpose to be abel to code smtg

quote_num = ds.shape[0]
dis_num = 4133

ds['disaster']=np.random.randint(0, dis_num, quote_num)
ds.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,disaster
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E,1894
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E,4026
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E,3386
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E,2927
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E,846


# Influencing factors in media coverage

Our hypothesis is that we can observe come connection between the volume of the appearance of a disaster in the media and the characteristics of the disasters. To be concrete we would expect that there are more quotes about disasters with more casualities. In this section our aim is to determine these factors in the observed data of diasaters and examine their contribution to media coverage.

## Reaching the disasters

As we have seen in the previous milestone, we can assume that to a specific disaster the quote realted to it will contain the name of the place and the type of the disaster. So our first goal is to assign the specific quotes according to this assumption to the adequate events.

Step 1: Collect the names and places of the biggest disasters.

In [27]:
disasters = pd.read_csv("datasets/emdat/emdat_public_2021_11_06_clean.csv")
disasters

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),human_readable_name,start_date,end_date
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,300000.0,3400000.0,NaN,3700000.0,11500000.0,200000.0,8000000.0,2010 Earthquake in Haiti,2010-01-01,2010-12-31
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,2010 Extreme temperature in Russian Federatio...,2010-01-01,2010-12-31
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,4000000.0,NaN,4000000.0,NaN,NaN,NaN,2010 Drought in Somalia,2010-01-01,2011-12-31
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,5933.0,362887.0,NaN,368820.0,NaN,37500000.0,210000000.0,2011 Earthquake in Japan,2011-01-01,2011-12-31
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,17932.0,5621790.0,NaN,5639722.0,NaN,100000.0,5174000.0,2015 Earthquake in Nepal,2015-01-01,2015-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,8839,2019-0415-TCD,Flood,NaN,NaN,Chad,TCD,Africa,False,NaN,...,NaN,423.0,NaN,423.0,NaN,NaN,NaN,2019 Flood in Chad,2019-01-01,2019-12-31
4129,8840,2019-0528-TON,Epidemic,Viral disease,Measles,Tonga,TON,Oceania,False,NaN,...,107.0,NaN,NaN,107.0,NaN,NaN,NaN,2019 Epidemic in Tonga,2019-01-01,2019-12-31
4130,8841,2019-0531-TON,Epidemic,Viral disease,Measles,Tonga,TON,Oceania,False,NaN,...,394.0,NaN,NaN,394.0,NaN,NaN,NaN,2019 Epidemic in Tonga,2019-01-01,2019-12-31
4131,8971,2019-0608-URY,Flood,NaN,NaN,Uruguay,URY,Americas,False,NaN,...,NaN,270.0,NaN,270.0,NaN,NaN,NaN,2019 Flood in Uruguay,2019-01-01,2019-12-31


Step 2: Collect the specific quotes



In [14]:
def contain_all_words_regex(words):
    return "".join([f"(?=.*{word})" for word in words])

def quotes_that_contain_all_words(words):
    return dataset[dataset.quotation.str.lower().str.contains(contain_all_words_regex(words), regex=True)]

## Determine the volume of media coverage

The best ways to measure the media coverage of a disaster is to count the number of occurrences in the quotes and the length between the first and last appearance. 

In this part we will calculate these indicators.

In [25]:
media_length = [] # last 
media_volume = [] # number of quotes
for i in range(dis_num): 
    d = ds[ds['disaster']==i]

    # calculate number of quotes
    v = d['numOccurrences'].values.sum()
    media_volume.append(v)

    # calculate length of appearance (in days)
    l = (d['date'].max() - d['date'].min()).days
    media_length.append(l)

disasters['Number of Quotes'] = media_volume
disasters['Length of Appearance'] = media_length

## Determine the contibuting factors

We should find first the relevant columns in the disaster dataset which can be tested that it has an effect of the volume numbers. 

In [75]:
# we have a dataset about the disasters with word developpment indicators
dis_wdi = pd.read_csv('wdi_with_disasters.csv')

indicators = dis_wdi['indicator_name'].unique()

for indi in indicators: 
    disasters[f'{indi}'] = np.ones(dis_num)


for i in tqdm(range(dis_num)): # iterate over disasters
    
    for indi in indicators: # iterate over indicators
     
        curr_dis = disasters.iloc[i]
        name = curr_dis['human_readable_name']
        year = int(curr_dis['start_date'][:4])

        val = dis_wdi[(dis_wdi['human_readable_name']==name) & (dis_wdi['indicator_name']==indi)][str(year)].values

        if len(val) == 0:
            pass
        else: 
            disasters.loc[disasters['human_readable_name']==name, f'{indi}'] = val[0]

            

disasters.head()

C:\Users\jhild\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 4133/4133 [1:37:34<00:00,  1.42s/it]  


,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,1.000000,1.00000,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


In [78]:
import pickle as pickle
with open(  f'disasters'+'.pickle', 'wb' ) as handle:
    pickle.dump(disasters, handle, protocol=pickle.HIGHEST_PROTOCOL)

disasters = pd.read_pickle('disasters.pickle')
disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,1.000000,1.00000,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


## Visualisations

This is the right place to make some nice visualizations for the datastory. 